In [5]:
import roamability as rb

import http
import re
from bs4 import BeautifulSoup
import sys

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type": "text/xml"})
    resp = client.getresponse()
    return resp.read()

def sri4sm(ogt, msisdn, dgt):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <sccp_np>7</sccp_np>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, dgt, msisdn, ogt)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

def sendSMS(ogt, imsi, msc, payload):
    req = """<ss7gw request=\"MTSMS_OTA\">
        <d_ssn>8</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <imsi>%s</imsi>
        <password>123</password>
        <sc_number>%s</sc_number>
        <key_id>1</key_id>
        <command>1</command>
        <corr_id>1</corr_id>
        <seed>1122</seed>
        <msisdn>972559300036</msisdn>
        <message>%s</message>
        <sms_imsi>%s</sms_imsi>
        <map>3</map>
        </ss7gw>""" % (ogt, msc, imsi, ogt, payload, imsi)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

In [28]:
# Encrypt plaintext to OTA payload

import math
from Crypto.Cipher import DES3

# Roamability Test keys example

key_hex_str = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
key = bytes.fromhex(key_hex_str)

# Plaintext

plaintext = '4E3F0E5A000000010018BC'
print(f'The original plaintext: {plaintext}')

# Split plaintext into 8 byte parts for DES3 CBC encryption

byte_array_list = []
for i in range(1, math.ceil(len(plaintext)/16)+1):
    hex_str = f'{plaintext[(i-1)*16:i*16]:0<16}'
    print(f'Plaintext 8 byte part {i}: {hex_str}')
    byte_array = bytes.fromhex(hex_str)
    byte_array_list.append(byte_array)

# Create cipher

cipher = DES3.new(key, DES3.MODE_CBC)

# Encrypt 8 byte parts

cipher_str = ''
for byte_array in byte_array_list:
    msg = cipher.encrypt(byte_array_list[0])
    for b in msg:
        cipher_str += f'{b:X}'
print(f'Encrypted:        {cipher_str}, len: {len(cipher_str)}')
example = '2028D0FC11B4CD2E540704D7775B065DA8'
print(f'Original example: {example}, len: {len(example)}')

# https://pycryptodome.readthedocs.io/en/latest/src/cipher/des3.html

The original plaintext: 4E3F0E5A000000010018BC
Plaintext 8 byte part 1: 4E3F0E5A00000001
Plaintext 8 byte part 2: 0018BC0000000000
Encrypted:        F89EFD3C839828BED542AFD9CB01AF1, len: 31
Original example: 2028D0FC11B4CD2E540704D7775B065DA8, len: 34


In [15]:
len('2028D0FC11B4CD2E540704D7775B065DA8')

34

In [35]:
checksum = 0
example = '2028D0FC11B4CD2E540704D7775B065DA8'
packet = bytes.fromhex(example[:-2])
# XOR checksum
for el in packet:
    checksum ^= el
print(hex(checksum))

0x95


In [10]:
# Make payload for SMS

ota_payload = '16FBF7FF7F1F0E0FCB4100001FE3869804070000000000'

as_rp = '0D0001'
as_mo = '0D0021'
header = '0000BFFF010000000000'
l = (len(ota_payload) + len(as_rp) + len(header))/2
sms_payload = f'02700000{int(l):X}{as_rp}0000BFFF010000000000{ota_payload}'
print(sms_payload)

02700000240D00010000BFFF01000000000016FBF7FF7F1F0E0FCB4100001FE3869804070000000000


In [11]:
# Send a SMS

# sms_payload = rb.get_info
sms_payload = ota_payload
# resp = sendSMS('97254120624', '425019613000536', '79112009927', sms_payload.upper()) # My test SIM
resp = sendSMS('550170002133', '724179999990002', '551184137200', sms_payload.upper())
BeautifulSoup(resp,'xml')

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response>

In [ ]:
# Decode response from SIM

s = BeautifulSoup(resp,'xml').response.response.text
rb.decode_payload_response(s)